In [ ]:
def remove_back(path):
  """
  Helper Function to remove Background
  str: path = path to image
  returns: newpath to image without Background
  """
  new_path = "img.png"
  !backgroundremover -i $path -o $new_path
  return new_path


def is_camel_black(path):
  """"
  Checks if camel is black.
  path: path to image.
  returns: True if Camel is black
  """
  from PIL import Image
  import imageio
  import os
  import numpy as np


  # remove background
  new_path = remove_back(path)
  # Read the created image
  img = imageio.imread(new_path, as_gray=True)
  # delete created files
  os.remove(new_path)
  # Get the mean of the image to check if it is black
  return np.mean(img) < 20

In [ ]:
def is_camel(image,is_camel_model):
    """"
    Checks if the input is a camel.
    image: image to check,
    is_camel_model: model to check with.
    """
    import tensorflow as tf
    # Prepocess input for resnet
    image = tf.keras.applications.resnet.preprocess_input(image)
    # predict
    pred = is_camel_model(image)
    # encode prediction
    is_camel = tf.keras.applications.resnet.decode_predictions(pred.numpy(), top=1)
    return is_camel[0][0][1] == "Arabian_camel"
    


In [ ]:
  def predict(paths, path_to_saved_model):
    """"
    Ranks camels with with a Neural Network.
    paths: paths to images to rank.
    returns: paths ordered
    """
    !pip install -q backgroundremover
    import numpy as np
    import tensorflow as tf
    import sys
    # Load in models so we do not need to load them in with every image
    is_camel_model = tf.keras.applications.resnet.ResNet152()
    model_classify = tf.keras.models.load_model(path_to_saved_model)
    ordered_dict = dict()
    for i, path in enumerate(paths):
      # load in image
      img = tf.keras.preprocessing.image.load_img(path)
      # resize it for out models
      img = tf.image.resize(np.array(img), (224,224))
      # put None in front for our models
      img = img[None,:,:]
      # Check if it is a camel
      if not is_camel(img, is_camel_model):
        print(f"Image number {i+1} is not a camel and will not be ranked" )
        continue
      if not is_camel_black(path):
        print(f"Camel number {i+1} is not black and will not be ranked" )
        continue
      # Run the img through the classifier and save the result. The minus has the effect that the first class is worth more
      ordered_dict[path] = - model_classify(img)[0][0]
    # order the dict
    ordered_dict = {k: v for k, v in sorted(ordered_dict.items(), key=lambda item: item[1], reverse = True)}
    print(ordered_dict)
    return list(ordered_dict.keys())

print(predict(["/content/drive/MyDrive/images_test/vlc_extracted00661.png", "/content/drive/MyDrive/images_test/vlc_extracted00651.png"],"/content/drive/MyDrive/saved_model/my_model_92%"))

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
/content/drive/MyDrive/images_test/vlc_extracted00661-background.png
2.3381927
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
/content/drive/MyDrive/images_test/vlc_extracted00651-background.png
0.98601913
{'/content/drive/MyDrive/images_test/vlc_extracted00651.png': <tf.Tensor: shape=(), dtype=float32, numpy=13.2826>, '/content/drive/MyDrive/images_test/vlc_extracted00661.png': <tf.Tensor: shape=(), dtype=float32, numpy=12.46401>}
['/content/drive/MyDrive/images_test/vlc_extrac